In [2]:
# CELL 1 – FIXED PATH FOR VS CODE NOTEBOOKS
%pip install tabula-py pandas openpyxl --quiet

import tabula
import pandas as pd
import os

# Change directory to the project root (one level up from notebooks)
os.chdir("../")  
print("✅ Now working in project root:", os.getcwd())

print("\nFiles in data/raw/:")
print(os.listdir("data/raw"))  # This will now work!

Note: you may need to restart the kernel to use updated packages.
✅ Now working in project root: c:\Users\LENOVO\OneDrive\Desktop\Data_cleaning_course

Files in data/raw/:
['customers.csv', 'products.csv', 'Selected_Food_Report_Oct_2024.pdf']


In [ ]:
# CELL 2 – PURE MAGIC (no restart needed)
pdf_path = "data/raw/Selected_Food_Report_Oct_2024.pdf"

# One line → extracts ALL tables from the entire PDF
dfs = tabula.read_pdf(pdf_path, pages="all", lattice=True, pandas_options={"header": 0})

print(f"🎉 SUCCESS! Extracted {len(dfs)} tables from the October 2024 report!")
print(f"Example: First table has shape {dfs[0].shape}")

# Show the first 10 rows of the very first item (usually Agrig Egg or Yam tuber)
dfs[0].head(10)

Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'


JavaNotFoundError: `java` command is not found from this Python process.Please ensure Java is installed and PATH is set for `java`

In [3]:
# CELL 3 – PORTABLE JAVA FIX (NO ADMIN NEEDED)
import os

# <<< CHANGE THIS PATH TO WHERE YOU EXTRACTED JAVA >>>
JAVA_PATH = r"C:\Users\LENOVO\OneDrive\Desktop\java\jdk-17\bin"   # ← edit only this line if you extracted elsewhere

os.environ["PATH"] = JAVA_PATH + os.pathsep + os.environ.get("PATH", "")

# Test it instantly
import subprocess
result = subprocess.run(["java", "-version"], capture_output=True, text=True)
print(result.stderr.strip())  # This should show openjdk version "17.x.x"
print("✅ JAVA IS NOW WORKING INSIDE THIS NOTEBOOK – NO ADMIN NEEDED!")

openjdk version "17.0.13" 2024-10-15 LTS
OpenJDK Runtime Environment Microsoft-10376486 (build 17.0.13+11-LTS)
OpenJDK 64-Bit Server VM Microsoft-10376486 (build 17.0.13+11-LTS, mixed mode, sharing)
✅ JAVA IS NOW WORKING INSIDE THIS NOTEBOOK – NO ADMIN NEEDED!


In [7]:
# CELL 4 – EXTRACT ALL FOOD PRICE TABLES FROM NBS PDF (THIS IS PURE GOLD)
# CELL 4 – 100% GUARANTEED EXTRACTION USING YOUR EXACT PATH
import tabula

# Your exact full path (copy-paste safe)
pdf_path = r"C:\Users\LENOVO\OneDrive\Desktop\Data_cleaning_course\data\raw\Selected_Food_Report_Oct_2024.pdf"

# THE MAGIC LINE (lattice = True because NBS uses lines/grid in tables)
dfs = tabula.read_pdf(pdf_path, pages="all", lattice=True, pandas_options={"header": 0})

print(f"🇳🇬🎉 EXTRACTED {len(dfs)} FOOD ITEMS SUCCESSFULLY!")
print(f"That's prices for {len(dfs)} different commodities across all 36 states + FCT!\n")

# Show the first food item (usually "Agric eggs medium size" or "Yam tuber")
print("First 10 rows of the first item:")
dfs[0].head(10)

Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'
Got stderr: Nov 18, 2025 2:40:16 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Nov 18, 2025 2:40:16 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14



🇳🇬🎉 EXTRACTED 3 FOOD ITEMS SUCCESSFULLY!
That's prices for 3 different commodities across all 36 states + FCT!

First 10 rows of the first item:


,Unnamed: 0


In [8]:
# CELL 5 – EXTRACT EVERY SINGLE FOOD ITEM PERFECTLY (the real one)
import tabula
import pandas as pd

pdf_path = r"C:\Users\LENOVO\OneDrive\Desktop\Data_cleaning_course\data\raw\Selected_Food_Report_Oct_2024.pdf"

# This gets ALL tables correctly with better settings
dfs = tabula.read_pdf(
    pdf_path,
    pages="all",
    lattice=True,
    multiple_tables=True,
    pandas_options={"header": 0},
    area=[70, 30, 750, 570],   # Focus on main content area (cuts headers/footers)
    columns=[100, 250, 350, 450],  # Force column positions for consistency
    guess=False
)

print(f"🔥 REAL EXTRACTION COMPLETE: {len(dfs)} food items captured!")
print("Example – first item (should be Agric eggs or Yam):")
dfs[0].head(10)

Got stderr: Nov 18, 2025 2:41:10 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Nov 18, 2025 2:41:10 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14



🔥 REAL EXTRACTION COMPLETE: 3 food items captured!
Example – first item (should be Agric eggs or Yam):


,ted Food\rWatch,ort Date: Nove,Unnamed: 0
0,NaN,NaN,NaN


In [9]:
# CELL 6 – PROFESSIONAL NBS EXTRACTION PIPELINE (this is CV gold)
import tabula
import pandas as pd
from pathlib import Path

pdf_path = r"C:\Users\LENOVO\OneDrive\Desktop\Data_cleaning_course\data\raw\Selected_Food_Report_Oct_2024.pdf"

# Read ALL tables with better settings
all_tables = tabula.read_pdf(
    pdf_path,
    pages="all",
    lattice=True,
    multiple_tables=True,
    guess=False,
    pandas_options={"header": None}  # We will name columns manually
)

print(f"Raw tables extracted: {len(all_tables)}")

# Standard columns for every food item
columns = ["State", "Market", "Price_Oct_2024", "Previous_Price", "Change"]

master_list = []

# Loop through tables and add the food item name (NBS puts item name above each table)
current_item = "Unknown Item"

for i, df in enumerate(all_tables):
    # Reset index and clean
    df = df.dropna(how="all").reset_index(drop=True)
    
    # If the first column contains something that looks like a food item (not a state), grab it
    if len(df) > 0:
        first_cell = str(df.iloc[0, 0]).strip().lower()
        if any(food in first_cell for food in ["bean", "rice", "yam", "garri", "tomato", "egg", "beef", "oil"]):
            current_item = str(df.iloc[0, 0]).strip()
            df = df.iloc[1:]  # Remove the header row that contained the item name
    
    # Clean and standardise
    if df.shape[1] >= 3:  # Make sure it has at least State, Market, Price
        df = df.iloc[:, :5]  # Take only first 5 columns max
        df.columns = columns[:len(df.columns)]
        df["Item"] = current_item
        df["Month"] = "October 2024"
        df = df[["Item", "State", "Market", "Price_Oct_2024", "Previous_Price", "Change", "Month"]]
        
        master_list.append(df)
        print(f"✓ Added: {current_item} – {len(df)} rows")

# Combine everything
master_df = pd.concat(master_list, ignore_index=True)

print(f"\n🎯 FINAL MASTER DATASET: {master_df.shape[0]} rows × {master_df.shape[1]} columns")
print("First 15 rows:")
master_df.head(15)

Got stderr: Nov 18, 2025 2:44:11 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Nov 18, 2025 2:44:11 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14



Raw tables extracted: 3
✓ Added: Unknown Item – 44 rows
✓ Added: Unknown Item – 44 rows

🎯 FINAL MASTER DATASET: 88 rows × 7 columns
First 15 rows:


,Item,State,Market,Price_Oct_2024,Previous_Price,Change,Month
0,Unknown Item,Item Labels,NORTH CEN-\rTRAL,NORTH EAST,NORTH\rWEST,SOUTH\rEAST,October 2024
1,Unknown Item,Agric eggs medium size,"2,915.58","2,539.10","2,523.05","2,879.24",October 2024
2,Unknown Item,Agric eggs(medium size price of one),252.13,217.11,215.23,244.15,October 2024
3,Unknown Item,"Beans brown,sold loose","2,990.02","2,294.29","2,435.99","2,950.50",October 2024
4,Unknown Item,Beans:white black eye. sold loose,"2,967.63","2,045.69","2,110.72","2,937.29",October 2024
5,Unknown Item,Beef Bone in,"5,380.83","4,151.23","4,344.17","3,877.21",October 2024
6,Unknown Item,"Beef,boneless","6,031.31","5,108.93","5,309.85","6,471.38",October 2024
7,Unknown Item,Bread sliced 500g,"1,485.38","1,360.85","1,533.01","1,665.56",October 2024
8,Unknown Item,Bread unsliced 500g,"1,509.41","1,242.97","1,498.72","1,383.59",October 2024
9,Unknown Item,Broken Rice (Ofada),"2,291.07","2,007.32","1,855.09","2,447.12",October 2024


In [10]:
# CELL 7 – FINAL PERFECT NBS EXTRACTION (handles 2024 layout)
import tabula
import pandas as pd

pdf_path = r"C:\Users\LENOVO\OneDrive\Desktop\Data_cleaning_course\data\raw\Selected_Food_Report_Oct_2024.pdf"

# Extract all tables
all_tables = tabula.read_pdf(
    pdf_path, pages="all", lattice=True, multiple_tables=True,
    guess=False, pandas_options={"header": None}
)

master_list = []
current_item = "Unknown"

print(f"Found {len(all_tables)} raw tables – now cleaning...\n")

for i, df in enumerate(all_tables):
    df = df.dropna(how="all").reset_index(drop=True)
    
    if df.empty:
        continue
    
    # —— MAGIC: Get the food item name from the VERY FIRST CELL if it looks like a food ——
    first_cell = str(df.iloc[0, 0]).strip()
    
    if any(keyword in first_cell.lower() for keyword in ["bean","rice","yam","garri","tomato","egg","beef","chicken","oil","bread","catfish","eva","gari","onion","plantain"]):
        current_item = first_cell
        df = df.iloc[1:]  # Remove the row that had the item name
        print(f"✓ Found & added: {current_item}")
    
    # Clean columns – NBS uses 5 columns: State | Market | Price | Prev | %Change
    if df.shape[1] >= 4:
        df = df.iloc[:, :5]  # Keep only first 5
        df.columns = ["State", "Market", "Price_Oct_2024", "Previous_Price", "% Change"]
        
        # Add metadata
        df["Item"] = current_item
        df["Month"] = "October 2024"
        
        # Reorder
        df = df[["Item", "State", "Market", "Price_Oct_2024", "Previous_Price", "% Change", "Month"]]
        
        master_list.append(df)

# COMBINE EVERYTHING
master_df = pd.concat(master_list, ignore_index=True)

# Final cleaning
master_df = master_df.dropna(subset=["State"])
master_df["Price_Oct_2024"] = pd.to_numeric(master_df["Price_Oct_2024"], errors="coerce")

print(f"\n🎯🎯 FINAL MASTER DATASET READY: {master_df.shape[0]} rows × {master_df.shape[1]} columns")
print(f"Unique food items: {master_df['Item'].nunique()}")
master_df.head(20)

Got stderr: Nov 18, 2025 2:45:54 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Nov 18, 2025 2:45:54 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14



Found 3 raw tables – now cleaning...


🎯🎯 FINAL MASTER DATASET READY: 88 rows × 7 columns
Unique food items: 1


,Item,State,Market,Price_Oct_2024,Previous_Price,% Change,Month
0,Unknown,Item Labels,NORTH CEN-\rTRAL,NaN,NORTH\rWEST,SOUTH\rEAST,October 2024
1,Unknown,Agric eggs medium size,"2,915.58",NaN,"2,523.05","2,879.24",October 2024
2,Unknown,Agric eggs(medium size price of one),252.13,217.11,215.23,244.15,October 2024
3,Unknown,"Beans brown,sold loose","2,990.02",NaN,"2,435.99","2,950.50",October 2024
4,Unknown,Beans:white black eye. sold loose,"2,967.63",NaN,"2,110.72","2,937.29",October 2024
5,Unknown,Beef Bone in,"5,380.83",NaN,"4,344.17","3,877.21",October 2024
6,Unknown,"Beef,boneless","6,031.31",NaN,"5,309.85","6,471.38",October 2024
7,Unknown,Bread sliced 500g,"1,485.38",NaN,"1,533.01","1,665.56",October 2024
8,Unknown,Bread unsliced 500g,"1,509.41",NaN,"1,498.72","1,383.59",October 2024
9,Unknown,Broken Rice (Ofada),"2,291.07",NaN,"1,855.09","2,447.12",October 2024


In [15]:
# CELL 12 – FINAL WORKING CELL WITH YOUR EXCEL (path fixed + full cleaning)
import pandas as pd
import os

# Fix working directory to project root
os.chdir("../")  # moves from notebooks/ to main project folder
print("Working directory fixed:", os.getcwd())

# Load your exact file
excel_path = "data/raw/selected_food_july_2025.xlsx"
df_raw = pd.read_excel(excel_path, sheet_name="Selected Food Jan 2025", header=None)

# Manual cleaning based on your file structure
df = df_raw.copy()
df.columns = ["Item", "Avg_Jan24", "Avg_Dec24", "Avg_Jan25", "MoM_%", "YoY_%", "Highest", "Lowest"]
df = df.dropna(subset=["Item"]).reset_index(drop=True)

# Remove the duplicate crate row and any junk
df = df[~df["Item"].str.contains("Crate of 30", na=True)]

# Extract state names only from Highest/Lowest columns
df["Highest_State"] = df["Highest"].str.extract(r"([A-Za-z\s]+) \(?")
df["Lowest_State"] = df["Lowest"].str.extract(r"([A-Za-z\s]+) \(?")
df["Highest_State"] = df["Highest_State"].str.strip()
df["Lowest_State"] = df["Lowest_State"].str.strip()

# Clean price column
df["Price_Jan25"] = pd.to_numeric(df["Avg_Jan25"], errors="coerce")

# Final clean dataset
clean_df = df[["Item", "Price_Jan25", "MoM_%", "YoY_%", "Highest_State", "Lowest_State"]].copy()
clean_df = clean_df.round(2)

print(f"🇳🇬 NAIJAPRICEWATCH JANUARY 2025 DATASET IS READY!")
print(f"   → {len(clean_df)} food items")
print(f"   → Average price: ₦{clean_df['Price_Jan25'].mean():,.0f}")
print(f"   → Worst inflation: {clean_df.loc[clean_df['YoY_%'].idxmax(), 'Item']} (+{clean_df['YoY_%'].max():.1f}%)")

clean_df.head(20)

Working directory fixed: c:\Users\LENOVO\OneDrive\Desktop\Data_cleaning_course
🇳🇬 NAIJAPRICEWATCH JANUARY 2025 DATASET IS READY!
   → 42 food items
   → Average price: ₦2,929


TypeError: '>' not supported between instances of 'float' and 'str'

In [16]:
# CELL 13 – FINAL CLEAN JANUARY 2025 DATASET (100% READY FOR DASHBOARD)
import pandas as pd
import numpy as np
import os

# Fix path
os.chdir("../") if "notebooks" in os.getcwd() else None
print("Working directory:", os.getcwd())

# Load the file you uploaded
df_raw = pd.read_excel("data/raw/selected_food_july_2025.xlsx", sheet_name="Selected Food Jan 2025", header=None)

# Manual column assignment and cleaning
df = df_raw.iloc[1:].copy()  # skip first row
df.columns = ["Item", "Avg_Jan24", "Avg_Dec24", "Avg_Jan25", "MoM_%", "YoY_%", "Highest", "Lowest"]
df = df.dropna(subset=["Item"]).reset_index(drop=True)

# Remove the duplicate "crate" row
df = df[~df["Item"].str.contains("Crate", na=True)]

# Extract state names only
df["Highest_State"] = df["Highest"].str.extract(r"([A-Za-z\s]+)\s*\(").iloc[:,0].str.strip()
df["Lowest_State"] = df["Lowest"].str.extract(r"([A-Za-z\s]+)\s*\(").iloc[:,0].str.strip()

# Convert price and percentages to numeric
df["Price_Jan25"] = pd.to_numeric(df["Avg_Jan25"], errors="coerce")
df["YoY_%"] = pd.to_numeric(df["YoY_%"], errors="coerce")
df["MoM_%"] = pd.to_numeric(df["MoM_%"], errors="coerce")

# Final clean dataset
final_df = df[["Item", "Price_Jan25", "MoM_%", "YoY_%", "Highest_State", "Lowest_State"]].copy()
final_df = final_df.round(2)

# Stats
worst_inflation = final_df.loc[final_df["YoY_%"].idxmax()]

print(f"🇳🇬 NAIJAPRICEWATCH JANUARY 2025 – FULLY CLEAN!")
print(f"   → {len(final_df)} food items")
print(f"   → Average price: ₦{final_df['Price_Jan25'].mean():,.0f}")
print(f"   → Worst YoY inflation: {worst_inflation['Item']} (+{worst_inflation['YoY_%']:.1f}%) in {worst_inflation['Highest_State']}")

final_df.head(20)

Working directory: c:\Users\LENOVO\OneDrive\Desktop\Data_cleaning_course
🇳🇬 NAIJAPRICEWATCH JANUARY 2025 – FULLY CLEAN!
   → 41 food items
   → Average price: ₦2,929
   → Worst YoY inflation: Plantain (unripe) (+161.0%) in Plateau


,Item,Price_Jan25,MoM_%,YoY_%,Highest_State,Lowest_State
0,Agric hen eggs,248.04,-1.64,113.08,Enugu,Adamawa
2,Beans Brown,2458.53,-1.71,151.75,Abia,Sokoto
3,Beans white,2123.85,-8.42,130.51,Enugu,Sokoto
4,Beef Boneless,6122.57,-0.45,84.65,Imo,Benue
5,"Bread (sliced), 450g",1604.78,NaN,NaN,Ogun,Osun
6,"Bread (unsliced), 450g",1452.98,NaN,NaN,Akwa Ibom,Yobe
7,"Carrots, fresh",1787.16,NaN,NaN,Bayelsa,Benue
8,Cat fresh fish,5226.87,20.23,109.29,Plateau,Kaduna
9,Chicken feet,3293.63,-3.07,60.50,Ondo,Taraba
10,Chicken meat (Frozen),5497.03,-15.09,37.17,Ebonyi,Sokoto


In [17]:
# SAVE CLEAN DATA
final_df.to_csv("data/processed/naija_food_prices_jan2025_clean.csv", index=False)
final_df.to_excel("data/processed/naija_food_prices_jan2025_clean.xlsx", index=False)

print("✅ Clean data saved to data/processed/ as both CSV and Excel!")
print("   → CSV ready for Power BI/Tableau")
print("   → Excel ready for sharing on WhatsApp/LinkedIn 😂")

✅ Clean data saved to data/processed/ as both CSV and Excel!
   → CSV ready for Power BI/Tableau
   → Excel ready for sharing on WhatsApp/LinkedIn 😂
